# Final Project: 17k Mobile Strategy Games Analysis
Samuel Folledo's final project for [DS1.1](https://github.com/Make-School-Courses/DS-1.1-Data-Analysis)

Thank you Tristan for the [17K Mobile Strategy Games
Every strategy game on the Apple App Store Dataset](https://www.kaggle.com/tristan581/17k-apple-app-store-strategy-games)